### Dedupe Example with NC Voter Data

Test using 1M rows (NC voter data file #1). 
Iteration 0: standard config for fields:
    fields = [
        {'field': 'givenname', 'type': 'String'},
        {'field': 'surname', 'type': 'String'},
        {'field': 'suburb', 'type': 'String'},
        {'field': 'postcode', 'type': 'String', 'has missing': True},
        ]
    results - no duplicates?
    
Iteration 1: use name type for given/surname fields fields:
    fields = [
        {'field': 'givenname', 'type': 'Name'},
        {'field': 'surname', 'type': 'Name'},
        {'field': 'suburb', 'type': 'String'},
        {'field': 'postcode', 'type': 'String', 'has missing': True},
        ]     

In [1]:
import os
import csv
import re
import logging
import optparse

import dedupe
from unidecode import unidecode

In [2]:
from future.utils import viewitems

import csv
import collections
import itertools

In [3]:
def preProcess(column):
    """
    Do a little bit of data cleaning with the help of Unidecode and Regex.
    Things like casing, extra spaces, quotes and new lines can be ignored.
    """
    column = unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    # If data is missing, indicate that by setting the value to `None`
    if not column:
        column = None
    return column

In [4]:
def readData(filename):
    """
    Read in our data from a CSV file and create a dictionary of records,
    where the key is a unique record ID and each value is dict
    """

    data_d = {}
    with open(filename) as f:
        reader = csv.DictReader(f)
        for row in reader:
            clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
            row_id = int(row['Id'])
            data_d[row_id] = dict(clean_row)

    return data_d

In [6]:
# ## Setup
file_path = '/Users/gizelleguerra/Documents/fedex_cip/dedupe_NC_exp0/'
input_file = 'NC1_clean.csv'
output_file = 'NCdd1_output_it1.csv'
settings_file = 'NCdd1_learned_settings'
training_file = 'NCdd1_training.json'

print('importing data ...')
data_d = readData(file_path + input_file)

importing data ...


In [10]:
# If a settings file already exists, we'll just load that and skip training
if os.path.exists(settings_file):
    print('reading from', settings_file)
    with open(settings_file, 'rb') as f:
        deduper = dedupe.StaticDedupe(f)
else:
    # ## Training

    # Define the fields dedupe will pay attention to
    fields = [
        {'field': 'givenname', 'type': 'Name'},
        {'field': 'surname', 'type': 'Name'},
        {'field': 'suburb', 'type': 'String'},
        {'field': 'postcode', 'type': 'String', 'has missing': True},
        ]

    # Create a new deduper object and pass our data model to it.
    deduper = dedupe.Dedupe(fields)

    # If we have training data saved from a previous run of dedupe,
    # look for it and load it in.
    # __Note:__ if you want to train from scratch, delete the training_file
    if os.path.exists(training_file):
        print('reading labeled examples from ', training_file)
        with open(training_file, 'rb') as f:
            deduper.prepare_training(data_d, f)
    else:
        deduper.prepare_training(data_d)

    # ## Active learning
    # Dedupe will find the next pair of records
    # it is least certain about and ask you to label them as duplicates
    # or not.
    # use 'y', 'n' and 'u' keys to flag duplicates
    # press 'f' when you are finished
    print('starting active labeling...')

    dedupe.console_label(deduper)

    # Using the examples we just labeled, train the deduper and learn
    # blocking predicates
    deduper.train()

    # When finished, save our training to disk
    with open(training_file, 'w') as tf:
        deduper.write_training(tf)

    # Save our weights and predicates to disk.  If the settings file
    # exists, we will skip all the training and learning next time we run
    # this file.
    with open(settings_file, 'wb') as sf:
        deduper.write_settings(sf)

INFO:dedupe.canopy_index:Removing stop word le
INFO:dedupe.canopy_index:Removing stop word an
INFO:dedupe.canopy_index:Removing stop word ar
INFO:dedupe.canopy_index:Removing stop word er
INFO:dedupe.canopy_index:Removing stop word ll
INFO:dedupe.canopy_index:Removing stop word en
INFO:dedupe.canopy_index:Removing stop word on
INFO:dedupe.canopy_index:Removing stop word in
INFO:dedupe.canopy_index:Removing stop word ma
INFO:dedupe.canopy_index:Removing stop word ri
INFO:dedupe.canopy_index:Removing stop word el
INFO:dedupe.canopy_index:Removing stop word le
INFO:dedupe.canopy_index:Removing stop word an
INFO:dedupe.canopy_index:Removing stop word ar
INFO:dedupe.canopy_index:Removing stop word er
INFO:dedupe.canopy_index:Removing stop word ll
INFO:dedupe.canopy_index:Removing stop word in
INFO:dedupe.canopy_index:Removing stop word on
INFO:dedupe.canopy_index:Removing stop word an
INFO:dedupe.canopy_index:Removing stop word ar
INFO:dedupe.canopy_index:Removing stop word ne
INFO:dedupe.c

starting active labeling...


 n


givenname : bonnie
surname : clay
suburb : raleigh
postcode : 27603

givenname : terry
surname : shinholser
suburb : raleigh
postcode : 27603

0/10 positive, 1/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 n


...

givenname : donald
surname : williams
suburb : charlotte
postcode : 28215

givenname : curtis
surname : williams
suburb : charlotte
postcode : 28208

4/10 positive, 13/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 n


givenname : rodney
surname : rorie
suburb : charlotte
postcode : 28213

givenname : rodney
surname : uhlenhake
suburb : charlotte
postcode : 28269

4/10 positive, 14/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 f


Finished labeling
INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
/Applications/anaconda3/lib/python3.6/site-packages/rlr/crossvalidation.py:122: RuntimeWarning: invalid value encountered in double_scalars
  * (true_distinct + false_distinct)))
INFO:rlr.crossvalidation:optimum alpha: 0.000010, score 0.15187688400251725
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(PartialPredicate: (wholeFieldPredicate, surname, Surname), SimplePredicate: (nearIntegersPredicate, postcode), SimplePredicate: (hundredIntegersOddPredicate, postcode))
INFO:dedupe.training:(SimplePredicate: (commonIntegerPredicate, postcode), TfidfNGramCanopyPredicate: (0.8, givenname), LevenshteinCanopyPredicate: (4, surname))


In [11]:
# ## Clustering

# `partition` will return sets of records that dedupe
# believes are all referring to the same entity.

print('clustering...')
clustered_dupes = deduper.partition(data_d, 0.5)

print('# duplicate sets', len(clustered_dupes))

clustering...


INFO:dedupe.canopy_index:Removing stop word in
INFO:dedupe.canopy_index:Removing stop word re
INFO:dedupe.canopy_index:Removing stop word ie
INFO:dedupe.canopy_index:Removing stop word on
INFO:dedupe.canopy_index:Removing stop word sh
INFO:dedupe.canopy_index:Removing stop word an
INFO:dedupe.canopy_index:Removing stop word ha
INFO:dedupe.canopy_index:Removing stop word na
INFO:dedupe.canopy_index:Removing stop word ra
INFO:dedupe.canopy_index:Removing stop word da
INFO:dedupe.canopy_index:Removing stop word el
INFO:dedupe.canopy_index:Removing stop word la
INFO:dedupe.canopy_index:Removing stop word er
INFO:dedupe.canopy_index:Removing stop word ri
INFO:dedupe.canopy_index:Removing stop word ne
INFO:dedupe.canopy_index:Removing stop word al
INFO:dedupe.canopy_index:Removing stop word en
INFO:dedupe.canopy_index:Removing stop word le
INFO:dedupe.canopy_index:Removing stop word ia
INFO:dedupe.canopy_index:Removing stop word is
INFO:dedupe.canopy_index:Removing stop word li
INFO:dedupe.b

KeyboardInterrupt: 

In [9]:
# ## Writing Results

# Write our original data back out to a CSV with a new column called
# 'Cluster ID' which indicates which records refer to each other.

cluster_membership = {}
for cluster_id, (records, scores) in enumerate(clustered_dupes):
    for record_id, score in zip(records, scores):
        cluster_membership[record_id] = {
            "Cluster ID": cluster_id,
            "confidence_score": score
        }

out_f = file_path + output_file
in_f = file_path + input_file

with open(out_f, 'w') as f_output, open(in_f) as f_input:

    reader = csv.DictReader(f_input)
    fieldnames = ['Cluster ID', 'confidence_score'] + reader.fieldnames

    writer = csv.DictWriter(f_output, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        row_id = int(row['Id'])
        row.update(cluster_membership[row_id])
        writer.writerow(row)

### Evaluation

In [19]:
def evaluateDuplicates(found_dupes, true_dupes):
    true_positives = found_dupes.intersection(true_dupes)
    false_positives = found_dupes.difference(true_dupes)
    uncovered_dupes = true_dupes.difference(found_dupes)

    print('found duplicate')
    print(len(found_dupes))

    print('precision')
    print(1 - len(false_positives) / float(len(found_dupes)))

    print('recall')
    print(len(true_positives) / float(len(true_dupes)))

In [20]:
def dupePairs(filename, rowname) :
    dupe_d = collections.defaultdict(list)

    with open(filename) as f:
        reader = csv.DictReader(f, delimiter=',', quotechar='"')
        for row in reader:
            dupe_d[row[rowname]].append(row['Id'])

    if 'x' in dupe_d :
        del dupe_d['x']

    dupe_s = set([])
    for (unique_id, cluster) in viewitems(dupe_d) :
        if len(cluster) > 1:
            for pair in itertools.combinations(cluster, 2):
                dupe_s.add(frozenset(pair))

    return dupe_s

In [21]:
manual_clusters = 'csv_example_input_with_true_ids.csv'
dedupe_clusters = 'csv_example_output.csv'

manual_clusters_fp = file_path + manual_clusters
dedupe_clusters_fp = file_path + dedupe_clusters

In [22]:
true_dupes = dupePairs(manual_clusters_fp, 'True Id')
test_dupes = dupePairs(dedupe_clusters_fp, 'Cluster ID')

In [23]:
evaluateDuplicates(test_dupes, true_dupes)

found duplicate
3087
precision
0.98607061872368
recall
0.46065375302663436
